In [ ]:
%matplotlib widget

import numpy as np
import k3d
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

from pyergonomics.importers import Unit, from_bvh

In [ ]:
#
# Load BVH file and extract hips XY position
#

UNIT = Unit.CM
BVH_FILE = "../examples/data/xsens/Case_Study_12-001.bvh"

settings = from_bvh(BVH_FILE, unit=UNIT, ignore_first_frame=True)
tracker = settings.tracker
skeleton = settings.pose_skeleton
fps = settings.frames_per_second
num_frames = settings.number_of_frames

print(settings)

# Extract hips XY position for all frames
person_id = tracker.get_person_ids()[0]
keypoints_dict = tracker.get_keypoints_3d_dict(person_id)
frames = sorted(keypoints_dict.keys())

hips_xy = np.array([
    [keypoints_dict[f][skeleton.hips][0], keypoints_dict[f][skeleton.hips][1]]
    for f in frames
])

print(f"Extracted {len(hips_xy)} frames of hips XY position")
print(f"XY range: X=[{hips_xy[:,0].min():.2f}, {hips_xy[:,0].max():.2f}], Y=[{hips_xy[:,1].min():.2f}, {hips_xy[:,1].max():.2f}]")

In [ ]:
#
# Rule-based movement classification functions
#

def classify_velocity(hips_xy, fps, thresholds=(0.1, 1.5)):
    """Approach A: Frame-by-frame instantaneous velocity classification."""
    displacement = np.diff(hips_xy, axis=0)
    velocity = np.linalg.norm(displacement, axis=1) * fps  # m/s
    velocity = np.insert(velocity, 0, 0)  # pad first frame
    
    labels = np.zeros(len(velocity), dtype=int)
    labels[velocity >= thresholds[0]] = 1  # walking
    labels[velocity >= thresholds[1]] = 2  # running
    return velocity, labels  # 0=stationary, 1=walking, 2=running


def classify_window(hips_xy, fps, window_sec=1.0, thresholds=(0.1, 1.5)):
    """Approach B: Sliding window displacement classification."""
    window_frames = int(window_sec * fps)
    n = len(hips_xy)
    displacement = np.zeros(n)
    
    for i in range(n):
        start = max(0, i - window_frames // 2)
        end = min(n, i + window_frames // 2)
        displacement[i] = np.linalg.norm(hips_xy[end-1] - hips_xy[start])
    
    speed = displacement / window_sec  # m/s equivalent
    labels = np.zeros(n, dtype=int)
    labels[speed >= thresholds[0]] = 1
    labels[speed >= thresholds[1]] = 2
    return speed, labels


def classify_hysteresis(hips_xy, fps, min_duration_sec=0.5, thresholds=(0.1, 1.5)):
    """Approach C: Velocity with hysteresis (minimum state duration)."""
    min_frames = int(min_duration_sec * fps)
    displacement = np.diff(hips_xy, axis=0)
    velocity = np.linalg.norm(displacement, axis=1) * fps
    velocity = np.insert(velocity, 0, 0)
    
    # Initial raw classification
    raw_labels = np.zeros(len(velocity), dtype=int)
    raw_labels[velocity >= thresholds[0]] = 1
    raw_labels[velocity >= thresholds[1]] = 2
    
    # Apply hysteresis: only change state after min_frames
    labels = np.copy(raw_labels)
    current_state = raw_labels[0]
    state_start = 0
    
    for i in range(1, len(raw_labels)):
        if raw_labels[i] != current_state:
            if i - state_start < min_frames:
                labels[i] = current_state  # keep old state
            else:
                current_state = raw_labels[i]
                state_start = i
    
    return velocity, labels

In [ ]:
#
# Run all three classification approaches
#

vel_a, labels_a = classify_velocity(hips_xy, fps)
vel_b, labels_b = classify_window(hips_xy, fps, window_sec=1.0)
vel_c, labels_c = classify_hysteresis(hips_xy, fps, min_duration_sec=0.5)

print("Classification results:")
print(f"  Instantaneous: stationary={np.sum(labels_a==0)}, walking={np.sum(labels_a==1)}, running={np.sum(labels_a==2)} frames")
print(f"  Window:        stationary={np.sum(labels_b==0)}, walking={np.sum(labels_b==1)}, running={np.sum(labels_b==2)} frames")
print(f"  Hysteresis:    stationary={np.sum(labels_c==0)}, walking={np.sum(labels_c==1)}, running={np.sum(labels_c==2)} frames")

In [ ]:
#
# Combined visualization: k3d + matplotlib + slider
#
from IPython.display import display

# --- k3d visualization ---
plot = k3d.plot(grid=(-2, -2, 0, 2, 2, 2), camera_mode='orbit')

# Spaghetti graph: root joint trajectory projected on floor
vertices = []
indices = []
for i, frame in enumerate(frames):
    pos = keypoints_dict[frame][skeleton.hips]
    vertices.append([pos[0], pos[1], 0])
    if i > 0 and frames[i] - frames[i-1] == 1:
        indices.append([len(vertices)-2, len(vertices)-1])

floor_trajectory = k3d.lines(vertices, indices, width=0.005, color=0x3366ff, indices_type="pair")
plot += floor_trajectory

# Skeleton joints and bones
joints_plot = k3d.points([], point_size=0.03, shader='flat', color=0xff0000)
lines_plot = k3d.lines([], list(skeleton.bones), width=0.005, color=0xff0000, indices_type="segment")
plot += joints_plot
plot += lines_plot

# --- Matplotlib (interactive with zoom/pan) ---
time_axis = np.arange(num_frames) / fps

# Use Output widget to contain matplotlib
mpl_output = widgets.Output()
with mpl_output:
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(time_axis, vel_a, label='Instantaneous Velocity', color='#3366ff', alpha=0.8, linewidth=0.8)
    ax.plot(time_axis, vel_b, label='Window Displacement', color='#ff6633', alpha=0.8, linewidth=1.2)
    # ax.plot(time_axis, vel_c, label='Hysteresis', color='#33cc33', alpha=0.8, linewidth=1.2)
    ax.axhline(y=0.1, color='#ffcc00', linestyle='--', alpha=0.5, label='Walk threshold (0.1 m/s)')
    ax.axhline(y=1.5, color='#ff4444', linestyle='--', alpha=0.5, label='Run threshold (1.5 m/s)')
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Speed (m/s)')
    ax.set_title('Movement Classification Comparison')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(bottom=0)
    playhead = ax.axvline(x=0, color='red', linewidth=2)
    plt.tight_layout()
    plt.show()

# --- Slider + sync function ---
def update_frame(n):
    joints = tracker.get_keypoints_for_person(person_id, frame=n)[0]
    joints_plot.positions = joints
    lines_plot.vertices = joints
    t = n / fps
    playhead.set_xdata([t, t])
    fig.canvas.draw_idle()

slider = widgets.IntSlider(
    min=0, max=num_frames - 1, step=1, value=0,
    description='Frame:', layout=widgets.Layout(width='100%')
)

# Display all together using VBox
plot.display()
display(widgets.VBox([mpl_output, slider]))
interact(update_frame, n=slider)

In [ ]:
#
# Classification summary
#

state_names = ['Stationary', 'Walking', 'Running']
colors = ['#99ff99', '#ffcc99', '#ff9999']

fig2, axes = plt.subplots(1, 3, figsize=(12, 4))

for ax, (title, labels) in zip(axes, [
    ('Instantaneous Velocity', labels_a),
    ('Window Displacement', labels_b),
    ('Hysteresis', labels_c)
]):
    counts = [np.sum(labels == i) for i in range(3)]
    total_sec = len(labels) / fps
    
    ax.pie(counts, labels=state_names, colors=colors, autopct='%1.1f%%', startangle=90)
    ax.set_title(f'{title}\n(Total: {total_sec:.1f}s)')

plt.tight_layout()
plt.show()